In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import os

In [3]:
def scrape_page(url, headers):
    try:
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            if response.status_code == 200:
                return response.content
            else:
                print("Failed to retrieve the page. Status code:", response.status_code)
                return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [ ]:
import pandas as pd
df=pd.read_csv('zomato_vesu.csv')
# restaurant_links = ['https://www.zomato.com/surat/la-pinoz-pizza-vesu/order']
for link in df['Restaurant URL'].iloc[21:28]:
    print(link)
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
    content = scrape_page(link, headers)
    if content:
            soup = BeautifulSoup(content, 'html.parser')
            name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
            main_driver = webdriver.Chrome()
            main_driver.get(link)
            catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-kUQWMX bmNHuj')]")
            cattext=[c.text for c in catgeory]
            # print(cattext)
            data=[]
            for cat in cattext :
                 h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                 section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")
                 cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")

                 for card in cards :
                    food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                    food_name = food_s.text
                    price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                    price = price_s.text
                    price=price[1:]
                    price=float(price)
                    p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                    p = p_s.text
                    veg_s = card.find_element(By.XPATH,"//div[contains(@class,'sc-1tx3445-0 kcsImg sc-1s0saks')]")
                    if veg_s.get_attribute('type') == 'veg':
                        veg = 1
                    else:
                         veg = 0
                    alco = 0
                    data.append([ cat, food_name, veg, alco, p, price])
                    df1 = pd.DataFrame(data, columns=['Category','Item_Name','Is_Veg(1:veg,0:non-veg)','Is_Alcohol(1:alcoholic,0:non-alcoholic)','Description','Price'])
                    df1.to_csv(f'{name}_Zomato_vesu.csv', index=False)
                    print(f"Data has been scraped and written to {name}_zomato_vesu.csv \n")
            main_driver.quit()

In [ ]:
#Surat

In [4]:
directory_path = "surat_areas_link/"
for fn in os.listdir(directory_path):
    
    file_path = os.path.join(directory_path, fn)
    # print(file_path[17:-4])
    
    df = pd.read_csv(file_path)
    try : 
        restaurant_links = df['Restaurant URL']
        for link in restaurant_links:
            print(link)
            headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
            content = scrape_page(link, headers)
            if content:
                soup = BeautifulSoup(content, 'html.parser')
                name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
                file_name = "surat_areas_link_menu/"+file_path[17:-4]+'_'+name+'.csv'
                if os.path.exists(file_name):
                    print(f"Already Scrapped for {file_name}")
                    continue                
                main_driver = webdriver.Chrome()
                main_driver.get(link)
                catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-')]")
                cattext=[c.text for c in catgeory]
                print(cattext)
                data=[]
                for cat in cattext :
                    try:
                        h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                        section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")
                        cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")

                        for card in cards :
                            food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                            food_name = food_s.text
                            print(food_name)
                            price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                            price = price_s.text
                            price=price[1:]
                            price=float(price)
                            print(price)
                            p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                            p = p_s.text
                            print(p)

                            veg_s = card.find_element(By.XPATH,".//div[contains(@class,'sc-1tx3445-0 kcsImg sc-1s0saks')]")
                            print(veg_s.get_attribute('type'))
                            if veg_s.get_attribute('type') == 'veg':
                                veg = 1
                            else:
                                veg = 0
                            alco = 0
                            data.append([ cat, food_name, veg, alco, p, price])
                            
                            df = pd.DataFrame(data, columns=['Category','Item_Name','Is_Veg(1:veg,0:non-veg)','Is_Alcohol(1:alcoholic,0:non-alcoholic)','Description','Price'])
                            df.to_csv(file_name, index=False)
                        print("Data has been scraped and written to "+file_name)

                    except Exception as e:
                        print(e)
                        continue
                
        # sub_driver.quit()
            main_driver.quit()
    except Exception as e :
        print(e)
        pass
    continue


https://www.zomato.com/surat/kfc-3-adajan-gam/order
Already Scrapped for surat_areas_link_menu/zomato_links_surat_Adajan Gam_KFC.csv
https://www.zomato.com/surat/vijay-dairy-sweets-adajan-gam/order
Already Scrapped for surat_areas_link_menu/zomato_links_surat_Adajan Gam_Vijay Dairy & Sweets.csv
https://www.zomato.com/surat/g-dada-adajan-gam/order
Already Scrapped for surat_areas_link_menu/zomato_links_surat_Adajan Gam_G Dada.csv
https://www.zomato.com/surat/famous-frankie-adajan-gam/order
Already Scrapped for surat_areas_link_menu/zomato_links_surat_Adajan Gam_Famous Frankie.csv
https://www.zomato.com/surat/shree-ganesh-aloopuri-rasawala-khaman-adajan-gam/order
[]
https://www.zomato.com/surat/the-glorious-restaurant-adajan-gam/order
Already Scrapped for surat_areas_link_menu/zomato_links_surat_Adajan Gam_The Glorious Restaurant.csv
https://www.zomato.com/surat/mahesh-pav-bhaji-1-adajan-gam/order
[]
https://www.zomato.com/surat/jakaas-chinese-food-adajan-gam/order
Already Scrapped for s

In [3]:
import pandas as pd
import os

In [15]:
directory_path = "surat_areas_link_menu/"

for fn in os.listdir(directory_path):
    if fn.endswith('.csv'):  
        file_path = os.path.join(directory_path, fn)
        
        df = pd.read_csv(file_path)
        
        excel_file_path = os.path.join(directory_path, fn.replace('.csv', '.xlsx'))
        
        df.to_excel(excel_file_path, index=False)

In [23]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory_path = "Zomato Restaurants/"

# Define the desired column order
desired_columns = ["Category", "Item_Name", "Description", "Is_Veg(1:veg,0:non-veg)", "Is_Alcohol(1:alcoholic,0:no alcoholic)", "Price"]

# Iterate through each file in the directory
for fn in os.listdir(directory_path):
    if fn.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(directory_path, fn)
        
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        df.columns=["Category","Item_Name","Is_Veg(1:veg,0:non-veg)","Is_Alcohol(1:alcoholic,0:no alcoholic)","Description","Price"]
        
        # Reorder the DataFrame columns
        df = df[desired_columns]
        
        # Define the new Excel file path
        excel_file_path = os.path.join(directory_path, fn.replace('.csv', '.xlsx'))
        
        # Save the DataFrame to an Excel file
        df.to_excel(excel_file_path, index=False)

print("Conversion from CSV to Excel completed with columns in the specified order.")


Conversion from CSV to Excel completed with columns in the specified order.


In [ ]:
#Coimbatore

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import os

def scrape_page(url, headers):
    try:
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            if response.status_code == 200:
                return response.content
            else:
                print("Failed to retrieve the page. Status code:", response.status_code)
                return None
    except Exception as e:
        print("An error occurred:", e)
        return None

directory_path = "/Users/Vansh Dalal/Documents/Programs/Zomato"
for fn in os.listdir(directory_path):    
    # print(fn)
    if fn.endswith('.xlsx') :
        file_path = os.path.join(directory_path, fn)
        print(file_path)
        # print(file_path[17:-4])
        
        df = pd.read_excel(file_path)
        try : 
            restaurant_links = df[['Restaurant URL','Name']]
            restaurant_links = restaurant_links[375:]
            for index,link in restaurant_links.iterrows():
                print(link['Restaurant URL'])
                headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
                content = scrape_page(link['Restaurant URL'], headers)
                if content:
                    soup = BeautifulSoup(content, 'html.parser')
                    # name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
                    name=link['Name']
                    desired_columns = ["Category", "Item_Name", "Description", "Is_Veg(1:veg,0:non-veg)", "Is_Alcohol(1:alcoholic,0:no alcoholic)", "Price"]
                    file_name = "zomato Coimbatore/Zomato_"+name+'.csv'
                    file_name1="zomato Coimbatore/Zomato_"+name+'.xlsx'
                    if os.path.exists(file_name1):
                        print(f"Already Scrapped for {file_name1}")
                        continue
                    if os.path.exists(file_name):
                        temp = pd.read_csv(file_name)
                        temp.columns=["Category","Item_Name","Is_Veg(1:veg,0:non-veg)","Is_Alcohol(1:alcoholic,0:no alcoholic)","Description","Price"]
                        temp = temp[desired_columns]
                        excel_file_path = os.path.join("coimbatur/", file_name.replace('.csv', '.xlsx'))
                        new_fileName = file_name.replace('.csv', '.xlsx')
                        temp.to_excel(new_fileName, index=False)
                        print(f"Already Scrapped for {new_fileName}")
                        continue                

                    main_driver = webdriver.Chrome()
                    main_driver.get(link['Restaurant URL'])
                    catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-')]")
                    print(len(catgeory))

                    cattext=[c.text for c in catgeory]
                    print(cattext)
                    data=[]
                    for cat in cattext :
                        try:
                            h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                            section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")
                            cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")

                            for card in cards :
                                food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                                food_name = food_s.text
                                print(food_name)
                                price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                                price = price_s.text
                                price=price[1:]
                                price=float(price)
                                print(price)
                                p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                                p = p_s.text
                                print(p)

                                veg_s = card.find_element(By.XPATH,".//div[contains(@class,'sc-1tx3445-0 kcsImg sc-1s0saks')]")
                                print(veg_s.get_attribute('type'))
                                if veg_s.get_attribute('type') == 'veg':
                                    veg = 1
                                else:
                                    veg = 0
                                alco = 0
                                data.append([ cat, food_name, p, veg, alco, price])
                                
                                df = pd.DataFrame(data, columns=['Category','Item_Name','Description','Is_Veg(1:veg,0:non-veg)','Is_Alcohol(1:alcoholic,0:no alcoholic)','Price'])
                                df.to_excel(file_name.replace('.csv', '.xlsx'), index=False)
                            print("Data has been scraped and written to "+file_name.replace('.csv', '.xlsx'))

                        except Exception as e:
                            print(e)
                            continue
                    
            # sub_driver.quit()
                main_driver.quit()
        except Exception as e :
            print(e)
            pass
        continue

/Users/Vansh Dalal/Documents/Programs/Zomato\Zomato_Coimbatore.xlsx
https://www.zomato.com/coimbatore/sri-balaji-arul-jothi-ramanathapuram/order
9
['Meals varieties', 'Starters', 'Main Course', 'Rice and Biryani', 'South Indian', 'Fried Rice and Noodles', 'Snacks', 'Drinks (Beverages)', 'Party Combos']
Meal
149.0
Rice, sambar, rasam, Buttermilk, special kulambu, kootu, poriyal, payasam, pickle, papad
veg
Sambar Rice
78.0

veg
2 Parotta
78.0

veg
Veg Biryani
111.0

veg
Lemon Rice
72.0

veg
Curd Rice
83.0

veg
Extra Curd
12.0

veg
Data has been scraped and written to zomato Coimbatore/Zomato_Sri Balaji Arul Jothi.xlsx
Chilli Gobi
189.0

veg
Chilli Mushroom
208.0
Juicy mushroom chunks dipped in seasoned batter, fried and then tossed in a ... read more
veg
Chilli Paneer
234.0
A dish, made from batter-coated paneer deep fried in oil and tossed in ... read more
veg
Chilli Baby Corn
221.0

veg
Mushroom Pepper Fry
221.0

veg
Paneer Pepper Fry
247.0

veg
Baby Corn Fry
189.0

veg
Mushroom Fry
18